In [1]:
import tensorflow as tf
import numpy as np

from konlpy.tag import Okt
import pandas as pd
import tensorflow as tf
import enum
import os
import re
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
FILTERS = "([~.,!?\"':;)(])"
PAD = "<PAD>"
STD = "<SOS>"
END = "<END>"
UNK = "<UNK>"

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(FILTERS)


DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'

PATH = 'data_in/ChatBotData.csv_short'
VOCAB_PATH = 'data_in/vocabulary.txt'

MAX_SEQUENCE = 10

## 데이터 불러오기

In [3]:
def load_data(path):
    # 판다스를 통해서 데이터를 불러온다.
    data_df = pd.read_csv(path, header=0)
    # 질문과 답변 열을 가져와 question과 answer에 넣는다.
    question, answer = list(data_df['Q']), list(data_df['A'])

    return question, answer

In [4]:
inputs, outputs = load_data(PATH)

## 토크나이징과 어휘사전 생성

In [5]:
def data_tokenizer(data):
    # 토크나이징 해서 담을 배열 생성
    words = []
    for sentence in data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 위 필터와 같은 값들을 정규화 표현식을
        # 통해서 모두 "" 으로 변환 해주는 부분이다.
        sentence = re.sub(CHANGE_FILTER, "", sentence)
        for word in sentence.split():
            words.append(word)
    # 토그나이징과 정규표현식을 통해 만들어진
    # 값들을 넘겨 준다.
    return [word for word in words if word]

def load_vocabulary(path, vocab_path):
    # 사전을 담을 배열 준비한다.
    vocabulary_list = []
    # 사전을 구성한 후 파일로 저장 진행한다.
    # 그 파일의 존재 유무를 확인한다.
    if not os.path.exists(vocab_path):
        # 이미 생성된 사전 파일이 존재하지 않으므로
        # 데이터를 가지고 만들어야 한다.
        # 그래서 데이터가 존재 하면 사전을 만들기 위해서
        # 데이터 파일의 존재 유무를 확인한다.
        if (os.path.exists(path)):
            # 데이터가 존재하니 판단스를 통해서
            # 데이터를 불러오자
            data_df = pd.read_csv(path, encoding='utf-8')
            # 판다스의 데이터 프레임을 통해서
            # 질문과 답에 대한 열을 가져 온다.
            question, answer = list(data_df['Q']), list(data_df['A'])
#             if DEFINES.tokenize_as_morph:  # 형태소에 따른 토크나이져 처리
#                 question = prepro_like_morphlized(question)
#                 answer = prepro_like_morphlized(answer)
            data = []
            # 질문과 답변을 extend을
            # 통해서 구조가 없는 배열로 만든다.
            data.extend(question)
            data.extend(answer)
            # 토큰나이져 처리 하는 부분이다.
            words = data_tokenizer(data)
            # 공통적인 단어에 대해서는 모두
            # 필요 없으므로 한개로 만들어 주기 위해서
            # set해주고 이것들을 리스트로 만들어 준다.
            words = list(set(words))
            # 데이터 없는 내용중에 MARKER를 사전에
            # 추가 하기 위해서 아래와 같이 처리 한다.
            # 아래는 MARKER 값이며 리스트의 첫번째 부터
            # 순서대로 넣기 위해서 인덱스 0에 추가한다.
            # PAD = "<PADDING>"
            # STD = "<START>"
            # END = "<END>"
            # UNK = "<UNKNWON>"
            words[:0] = MARKER
        # 사전을 리스트로 만들었으니 이 내용을
        # 사전 파일을 만들어 넣는다.
        with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:
            for word in words:
                vocabulary_file.write(word + '\n')

    # 사전 파일이 존재하면 여기에서
    # 그 파일을 불러서 배열에 넣어 준다.
    with open(vocab_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())

    # 배열에 내용을 키와 값이 있는
    # 딕셔너리 구조로 만든다.
    char2idx, idx2char = make_vocabulary(vocabulary_list)
    # 두가지 형태의 키와 값이 있는 형태를 리턴한다.
    # (예) 단어: 인덱스 , 인덱스: 단어)
    return char2idx, idx2char, len(char2idx)


def make_vocabulary(vocabulary_list):
    # 리스트를 키가 단어이고 값이 인덱스인
    # 딕셔너리를 만든다.
    char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}
    # 리스트를 키가 인덱스이고 값이 단어인
    # 딕셔너리를 만든다.
    idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}
    # 두개의 딕셔너리를 넘겨 준다.
    return char2idx, idx2char

In [6]:
char2idx, idx2char, vocab_size = load_vocabulary(PATH, VOCAB_PATH)

## 학습 데이터 생성

In [7]:
def enc_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다.)
    sequences_input_index = []
    # 하나의 인코딩 되는 문장의
    # 길이를 가지고 있다.(누적된다.)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
#     if DEFINES.tokenize_as_morph:
#         value = prepro_like_morphlized(value)

    print(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 인코딩 할때
        # 가지고 있기 위한 배열이다.
        sequence_index = []
        # 문장을 스페이스 단위로
        # 자르고 있다.
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고
            # 그 값을 가져와 sequence_index에 추가한다.
            if dictionary.get(word) is not None:
                sequence_index.extend([dictionary[word]])
            # 잘려진 단어가 딕셔너리에 존재 하지 않는
            # 경우 이므로 UNK(2)를 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을
        # sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과
    # 그 길이를 넘겨준다.
    return np.asarray(sequences_input_index), sequences_length

In [8]:
def dec_output_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다)
    sequences_output_index = []
    # 하나의 디코딩 입력 되는 문장의
    # 길이를 가지고 있다.(누적된다)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
#     if DEFINES.tokenize_as_morph:
#         value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 디코딩 할때 가지고
        # 있기 위한 배열이다.
        sequence_index = []
        # 디코딩 입력의 처음에는 START가 와야 하므로
        # 그 값을 넣어 주고 시작한다.
        # 문장에서 스페이스 단위별로 단어를 가져와서 딕셔너리의
        # 값인 인덱스를 넣어 준다.
        sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을
        # sequences_output_index 넣어 준다.
        sequences_output_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_output_index), sequences_length

In [9]:
def dec_target_processing(value, dictionary):
    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다)
    sequences_target_index = []
    # 형태소 토크나이징 사용 유무
#     if DEFINES.tokenize_as_morph:
#         value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE - 1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을
        # sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index)

In [10]:
index_inputs, input_seq_len = enc_processing(inputs, char2idx)
index_outputs, output_seq_len = dec_output_processing(outputs, char2idx)
index_targets = dec_target_processing(outputs, char2idx)

['가끔 궁금해', '가끔 뭐하는지 궁금해', '가끔은 혼자인게 좋다', '가난한 자의 설움', '가만 있어도 땀난다', '가상화폐 쫄딱 망함', '가스불 켜고 나갔어', '가스불 켜놓고 나온거 같아', '가스비 너무 많이 나왔다.', '가스비 비싼데 감기 걸리겠어', '남자친구 교회 데려가고 싶어', '남자친구 또 운동 갔어', '남자친구 생일인데 뭘 줄까', '남자친구 승진 선물로 뭐가 좋을까?', '남자친구 오늘 따라 훈훈해 보인다', '남자친구 오늘 좀 질린다.', '남자친구가 나 안 믿어줘', '남자친구가 너무 바빠', '남자친구가 너무 운동만 해', '남자친구가 너무 잘생겼어']


In [11]:
# Show length
print(len(index_inputs), len(input_seq_len), len(index_outputs), len(output_seq_len), len(index_targets))

20 20 20 20 20


## Create a tf.data dataset

In [12]:
MODEL_NAME = 'seq2seq_kor'
BATCH_SIZE = 2
EPOCH = 100
UNITS = 1024
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.1 

In [13]:
class Encoder(tf.keras.layers.Layer):
    #def __init__(self, **kargs):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size 
        self.embedding_dim = embedding_dim          
        
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self, batch_sz=None):
        if batch_sz is None:
            batch_sz = self.batch_sz
        return tf.zeros((batch_sz, self.enc_units))

In [14]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)

        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [15]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.vocab_size = vocab_size 
        self.embedding_dim = embedding_dim  
        
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(self.vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)
        
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = self.embedding(x)

        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
            
        x = self.fc(output)
        
        return x, state, attention_weights

In [16]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

def accuracy(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [17]:
class Seq2seq(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, dec_units, batch_sz, end_token_idx=2):    
        super(Seq2seq, self).__init__()
        self.end_token_idx = end_token_idx
        self.encoder = Encoder(vocab_size, embedding_dim, enc_units, batch_sz) #vocab_size, embedding_dim, enc_units, batch_sz
        self.decoder = Decoder(vocab_size, embedding_dim, dec_units, batch_sz) #vocab_size, embedding_dim, dec_units, batch_sz

    def call(self, x):
        inp, tar = x

        enc_hidden = self.encoder.initialize_hidden_state()
        enc_output, enc_hidden = self.encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        #dec_input = tf.expand_dims([STD_INDEX] * BATCH_SIZE, 1)
        predict_tokens = list()
        for t in range(0, tar.shape[1]):
            dec_input = tf.dtypes.cast(tf.expand_dims(tar[:, t], 1), tf.float32) 
            predictions, dec_hidden, _ = self.decoder(dec_input, dec_hidden, enc_output)
            predict_tokens.append(tf.dtypes.cast(predictions, tf.float32))
            # using teacher forcing
            #dec_input = tf.dtypes.cast(tf.expand_dims(tar[:, t], 1), tf.float32)      
        return tf.stack(predict_tokens, axis=1)
    
    # evaluate 함수 대신 이 함수를 활용할 수 있음
    def inference(self, x):
        inp  = x

        enc_hidden = self.encoder.initialize_hidden_state(batch_sz=1)
        enc_output, enc_hidden = self.encoder(inp, enc_hidden)

        dec_hidden = enc_hidden
        index_outputs, output_seq_len = dec_output_processing(outputs, char2idx)
        dec_input = tf.expand_dims([STD_INDEX], 1)
        predict_tokens = list()
        for t in range(0, MAX_SEQUENCE):
            predictions, dec_hidden, _ = self.decoder(dec_input, dec_hidden, enc_output)
            predict_token = tf.argmax(predictions[0])
            if predict_token == self.end_token_idx:
                break
            predict_tokens.append(predict_token)
            dec_input = tf.dtypes.cast(tf.expand_dims([predict_token], 0), tf.float32)      
        return tf.stack(predict_tokens, axis=0).numpy()

In [18]:
model = Seq2seq(vocab_size, EMBEDDING_DIM, UNITS, UNITS, BATCH_SIZE)

In [19]:
model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(1e-3), metrics=[accuracy])

In [20]:
PATH = DATA_OUT_PATH + MODEL_NAME
if not(os.path.isdir(PATH)):
        os.makedirs(os.path.join(PATH))
        
checkpoint_path = DATA_OUT_PATH + MODEL_NAME + '/weights.{epoch:02d}-{val_accuracy:.2f}.h5'
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=1)

history = model.fit([index_inputs, index_outputs], index_targets,
                    batch_size=BATCH_SIZE, epochs=EPOCH,
                    validation_split=VALIDATION_SPLIT, callbacks=[earlystop_callback, cp_callback])

Train on 18 samples, validate on 2 samples
Epoch 1/100
16/18 [=========================>....] - ETA: 2s - loss: 2.0837 - accuracy: 0.6373
Epoch 00001: val_accuracy improved from -inf to 0.63500, saving model to ./data_out/seq2seq_kor/weights.01-0.63.h5
18/18 [==============================] - 20s 1s/sample - loss: 2.1041 - accuracy: 0.6363 - val_loss: 1.7220 - val_accuracy: 0.6350
Epoch 2/100
16/18 [=========================>....] - ETA: 0s - loss: 1.8517 - accuracy: 0.6352
Epoch 00002: val_accuracy improved from 0.63500 to 0.64000, saving model to ./data_out/seq2seq_kor/weights.02-0.64.h5
18/18 [==============================] - 2s 118ms/sample - loss: 1.8964 - accuracy: 0.6354 - val_loss: 1.5685 - val_accuracy: 0.6400
Epoch 3/100
16/18 [=========================>....] - ETA: 0s - loss: 1.6766 - accuracy: 0.6453
Epoch 00003: val_accuracy improved from 0.64000 to 0.64167, saving model to ./data_out/seq2seq_kor/weights.03-0.64.h5
18/18 [==============================] - 2s 117ms/sample 

Epoch 25/100
16/18 [=========================>....] - ETA: 0s - loss: 0.3199 - accuracy: 0.8214
Epoch 00025: val_accuracy improved from 0.82000 to 0.82340, saving model to ./data_out/seq2seq_kor/weights.25-0.82.h5
18/18 [==============================] - 2s 117ms/sample - loss: 0.3078 - accuracy: 0.8216 - val_loss: 0.4743 - val_accuracy: 0.8234
Epoch 26/100
16/18 [=========================>....] - ETA: 0s - loss: 0.1780 - accuracy: 0.8255
Epoch 00026: val_accuracy improved from 0.82340 to 0.82769, saving model to ./data_out/seq2seq_kor/weights.26-0.83.h5
18/18 [==============================] - 2s 116ms/sample - loss: 0.1785 - accuracy: 0.8257 - val_loss: 0.5391 - val_accuracy: 0.8277
Epoch 27/100
16/18 [=========================>....] - ETA: 0s - loss: 0.1391 - accuracy: 0.8297
Epoch 00027: val_accuracy improved from 0.82769 to 0.83222, saving model to ./data_out/seq2seq_kor/weights.27-0.83.h5
18/18 [==============================] - 2s 118ms/sample - loss: 0.1311 - accuracy: 0.8299 -

Epoch 49/100
16/18 [=========================>....] - ETA: 0s - loss: 0.0010 - accuracy: 0.9007
Epoch 00049: val_accuracy improved from 0.89979 to 0.90163, saving model to ./data_out/seq2seq_kor/weights.49-0.90.h5
18/18 [==============================] - 2s 117ms/sample - loss: 0.0011 - accuracy: 0.9008 - val_loss: 1.1749 - val_accuracy: 0.9016
Epoch 50/100
16/18 [=========================>....] - ETA: 0s - loss: 0.0010 - accuracy: 0.9025    
Epoch 00050: val_accuracy improved from 0.90163 to 0.90340, saving model to ./data_out/seq2seq_kor/weights.50-0.90.h5
18/18 [==============================] - 2s 116ms/sample - loss: 0.0010 - accuracy: 0.9026 - val_loss: 1.1810 - val_accuracy: 0.9034
Epoch 51/100
16/18 [=========================>....] - ETA: 0s - loss: 9.9470e-04 - accuracy: 0.9043
Epoch 00051: val_accuracy improved from 0.90340 to 0.90510, saving model to ./data_out/seq2seq_kor/weights.51-0.91.h5
18/18 [==============================] - 2s 110ms/sample - loss: 9.6809e-04 - accura

16/18 [=========================>....] - ETA: 0s - loss: 4.7639e-04 - accuracy: 0.9295
Epoch 00072: val_accuracy improved from 0.92901 to 0.92986, saving model to ./data_out/seq2seq_kor/weights.72-0.93.h5
18/18 [==============================] - 2s 116ms/sample - loss: 4.8594e-04 - accuracy: 0.9295 - val_loss: 1.2729 - val_accuracy: 0.9299
Epoch 73/100
16/18 [=========================>....] - ETA: 0s - loss: 4.9907e-04 - accuracy: 0.9303
Epoch 00073: val_accuracy improved from 0.92986 to 0.93068, saving model to ./data_out/seq2seq_kor/weights.73-0.93.h5
18/18 [==============================] - 2s 117ms/sample - loss: 4.7528e-04 - accuracy: 0.9303 - val_loss: 1.2778 - val_accuracy: 0.9307
Epoch 74/100
16/18 [=========================>....] - ETA: 0s - loss: 4.4587e-04 - accuracy: 0.9311
Epoch 00074: val_accuracy improved from 0.93068 to 0.93149, saving model to ./data_out/seq2seq_kor/weights.74-0.93.h5
18/18 [==============================] - 2s 122ms/sample - loss: 4.6199e-04 - accurac

16/18 [=========================>....] - ETA: 0s - loss: 2.8477e-04 - accuracy: 0.9442
Epoch 00095: val_accuracy improved from 0.94394 to 0.94442, saving model to ./data_out/seq2seq_kor/weights.95-0.94.h5
18/18 [==============================] - 2s 115ms/sample - loss: 2.9515e-04 - accuracy: 0.9442 - val_loss: 1.3288 - val_accuracy: 0.9444
Epoch 96/100
16/18 [=========================>....] - ETA: 0s - loss: 2.9220e-04 - accuracy: 0.9447
Epoch 00096: val_accuracy improved from 0.94442 to 0.94490, saving model to ./data_out/seq2seq_kor/weights.96-0.94.h5
18/18 [==============================] - 2s 115ms/sample - loss: 2.8991e-04 - accuracy: 0.9447 - val_loss: 1.3305 - val_accuracy: 0.9449
Epoch 97/100
16/18 [=========================>....] - ETA: 0s - loss: 2.8615e-04 - accuracy: 0.9452
Epoch 00097: val_accuracy improved from 0.94490 to 0.94536, saving model to ./data_out/seq2seq_kor/weights.97-0.95.h5
18/18 [==============================] - 2s 109ms/sample - loss: 2.8478e-04 - accurac

### Evaluate

In [22]:
SAVE_FILE_NM = "weights.100-0.95.h5"
model.load_weights(os.path.join(DATA_OUT_PATH, MODEL_NAME, SAVE_FILE_NM))

In [23]:
# 방법 2: 모델 내 inference 함수를 활용한 출력

query = "남자친구 승진 선물로 뭐가 좋을까?"

test_index_inputs, _ = enc_processing([query], char2idx)    
predict_tokens = model.inference(test_index_inputs)

print(' '.join([idx2char[t] for t in predict_tokens]))

['남자친구 승진 선물로 뭐가 좋을까?']
평소에 필요했던 게 좋을 것 같아요
